In [1]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

In [2]:


# ----------------------------
# EXCEL'DEN PARAMETRELERİN OLDUĞU DOSYAYI OKUMA
# ----------------------------
data_path = "C:/Users/Mehmet/Desktop/parametre10.xlsx"
xls = pd.ExcelFile(data_path)



In [3]:
f_k = {"R": 3, "M": 2} # R ve M ekiplerinin sabir yer değiştirme maliyeti

In [4]:
# Sets sayfasından kümeleri çek
df_sets = pd.read_excel(xls, sheet_name="Sets")
I = df_sets[df_sets["Set"] == "I"]["Element"].str.strip().str.upper().tolist()
J = df_sets[df_sets["Set"] == "J"]["Element"].str.strip().str.upper().tolist()
D = df_sets[df_sets["Set"] == "D"]["Element"].tolist()
K = df_sets[df_sets["Set"] == "K"]["Element"].str.strip().str.upper().tolist()

# Diğer parametre sayfaları
swi_df = pd.read_excel(xls, sheet_name="SWIjd")
dij_df = pd.read_excel(xls, sheet_name="dij")
cijk_df = pd.read_excel(xls, sheet_name="cijk")


In [5]:


# ----------------------------
# sidk'nin gün bazlı oransal hesaplanması. tedarik illerinde bulunan R M E değerlerini gün bazlı olarak oransal şekilde dağıttık
# ----------------------------
ratios = {
    1: 0.30,
    2: 0.30,
    3: 0.20,
    4: 0.1,
    5: 0.1
}
total_capacity = {"R": 24000, "M": 18000, "E": 15000}
supply_cities = df_sets[df_sets["Set"] == "I"]["Element"].str.strip().str.upper().unique()

records = []
for city in supply_cities:
    for day in ratios:
        row = {"i": city, "d": day}
        for k in total_capacity:
            row[k] = int(total_capacity[k] * ratios[day])
        records.append(row)

sidk_df = pd.DataFrame(records)
sidk = {(row.i, row.d, k): getattr(row, k) for row in sidk_df.itertuples(index=False) for k in ["R", "M", "E"]}


In [6]:


# ----------------------------
# Parametre sözlükleri
# ----------------------------
cijk_df["i"] = cijk_df["i"].str.strip().str.upper()
cijk_df["j"] = cijk_df["j"].str.strip().str.upper()
cijk_df["k"] = cijk_df["k"].str.strip().str.upper()
cijk = {(row.i, row.j, row.k): row.cost for row in cijk_df.itertuples(index=False)}

SWIjd = {(row.j.strip().upper(), row.d): row.SWIjd for row in swi_df.itertuples(index=False)}
dij = {(row.i.strip().upper(), row.j.strip().upper()): row.distance for row in dij_df.itertuples(index=False)}

# ----------------------------
# MODEL INIT
# ----------------------------
model = Model("Disaster_Team_Allocation")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2436270
Academic license 2436270 - for non-commercial use only - registered to me___@hacettepe.edu.tr


In [7]:
IJ_valid = set((i, j, k) for (i, j, k) in cijk.keys())

x = model.addVars(
    ((i, j, d, k) for (i, j, k) in IJ_valid for d in D),
    vtype=GRB.INTEGER, name="x"
)

m = model.addVars(I, J, J, D[:-1], ["R", "M"], vtype=GRB.INTEGER, name="m")
lambdak = model.addVars(["R", "M"], vtype=GRB.CONTINUOUS, lb=0.0, name="lambda")

In [8]:



# ----------------------------
# OBJECTIVE FUNCTION
# ----------------------------


assignment_cost = quicksum(
    cijk[i, j, k] * dij[i, j] * x[i, j, d, k]
    for (i, j, k) in IJ_valid for d in D
)


movement_cost = quicksum(
    (dij[j, jp] + f_k[k]) * m[i, j, jp, d, k]
    for i in I for j in J for jp in J for d in D[:-1] for k in ["R", "M"]
    if (j, jp) in dij
)

model.setObjective(assignment_cost + movement_cost, GRB.MINIMIZE)

In [9]:
# ----------------------------
# CONSTRAINTS
# ----------------------------
for i in I:                                                                             #(2)
    for d in D:
        for k in K:
            model.addConstr(
                quicksum(x[i, j, d, k] for j in J if (i, j, k) in IJ_valid) == sidk[i, d, k],
                name=f"supply_{i}_{d}_{k}"
            )

for i in I:                                                                             #(3)
    for d in D[:-1]:
        for k in ["R", "M"]:
            model.addConstr(
                quicksum(x[i, j, d + 1, k] for j in J if (i, j, k) in IJ_valid) ==
                quicksum(x[i, j, d, k] for j in J if (i, j, k) in IJ_valid),
                name=f"continuity_{i}_{d}_{k}"
            )

for i in I:                                                                            #(4)
    for j in J:
        for d in D[:-1]:
            for k in ["R", "M"]:
                if (i, j, k) in IJ_valid:
                    model.addConstr(
                        quicksum(m[i, j, jp, d, k] for jp in J) <= x[i, j, d, k],
                        name=f"move_from_presence_{i}_{j}_{d}_{k}"
                    )

for i in I:                                                                           #(5)
    for jp in J:
        for d in D[:-1]:
            for k in ["R", "M"]:
                if any((i, j, k) in IJ_valid for j in J):
                    if (i, jp, k) in IJ_valid:
                        model.addConstr(
                        x[i, jp, d + 1, k] >= quicksum(m[i, j, jp, d, k] for j in J),
                        name=f"move_result_assignment_{i}_{jp}_{d}_{k}"
                    )

for j in J:                                                                           #(6)
    for d in D:
        for k in ["R", "M"]:
            model.addConstr(
                quicksum(x[i, j, d, k] for i in I if (i, j, k) in IJ_valid) == lambdak[k] * SWIjd[j, d],
                name=f"swi_allocation_{j}_{d}_{k}"
            )


for j in J:                                                                           #(7)
    for d in D:
        model.addConstr(
           quicksum(x[i, j, d, "R"] for i in I if (i, j, "R") in IJ_valid) <=
           3 * quicksum(x[i, j, d, "M"] for i in I if (i, j, "M") in IJ_valid),
           name=f"medical_support_ratio_{j}_{d}"
      )
        
for j in J:                                                                          #(8)
    for d in D:
        model.addConstr(
            quicksum(x[i, j, d, "R"] for i in I if (i, j, "R") in IJ_valid) <=
            5 * quicksum(x[i, j, D[0], "E"] for i in I if (i, j, "E") in IJ_valid),
            name=f"equipment_support_ratio_{j}_{d}"
        )

for i in I:                                                                         #(9)
    for j in J:
        for d in D[1:]:
            if (i, j, "E") in IJ_valid:
                model.addConstr(
                    x[i, j, d, "E"] == x[i, j, D[0], "E"],
                    name=f"equipment_fixed_{i}_{j}_{d}"
                )
                
                
for j in J:                                                                         #(10)
    for d in D:
        model.addConstr(
            quicksum(x[i, j, d, "R"] for i in I if (i, j, "R") in IJ_valid)
            <= quicksum(x[i, j, d2, "E"] for i in I for d2 in D if d2 <= d and (i, j, "E") in IJ_valid),
            name=f"order_E_before_R_{j}_{d}"
        )

        
for j in J:                                                                          #(11)
    for d in D:
        model.addConstr(
            quicksum(x[i, j, d, "M"] for i in I if (i, j, "M") in IJ_valid)
            <= quicksum(x[i, j, d2, "R"] for i in I for d2 in D if d2 <= d and (i, j, "R") in IJ_valid),
            name=f"order_R_before_M_{j}_{d}"
        )


# ----------------------------
# SOLVE
# ----------------------------
model.optimize()


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-4712MQ CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2436270 - for non-commercial use only - registered to me___@hacettepe.edu.tr
Optimize a model with 7857 rows, 38524 columns and 99048 nonzeros
Model fingerprint: 0x2c103062
Variable types: 2 continuous, 38522 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-02, 5e+00]
  Objective range  [2e+00, 6e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 7e+03]
Presolve removed 6900 rows and 38184 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [10]:
if model.status == GRB.INFEASIBLE:
    print("Model infeasible. Computing IIS...")
    model.computeIIS()
    model.write("infeasible.ilp")

Model infeasible. Computing IIS...
Academic license 2436270 - for non-commercial use only - registered to me___@hacettepe.edu.tr

Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0      7792         -         0     38524         -           0s
       44        44        44         0         0         0           2s

IIS computed: 44 constraints, 0 bounds
IIS runtime: 1.95 seconds (0.10 work units)
